# Colab Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
"""
Change directory to where this file is located
"""
#%cd '/content/drive/MyDrive/DLSP/Drna-master'

'\nChange directory to where this file is located\n'

# Unzipping weights


In [ ]:
zip_path_weights = "/content/drive/MyDrive/DLSP/models/BestModelSoFar.zip"
!cp "{zip_path_weights}" .
!unzip -q BestModelSoFar.zip
!rm BestModelSoFar.zip

cp: cannot stat '/content/drive/MyDrive/DLSP/models/BestModelSoFar.zip': No such file or directory
unzip:  cannot find or open BestModelSoFar.zip, BestModelSoFar.zip.zip or BestModelSoFar.zip.ZIP.
rm: cannot remove 'BestModelSoFar.zip': No such file or directory


# Util functions

In [3]:
import os
import copy
import numpy as np
from torch.autograd import Variable
import torch
from torch.optim import SGD
from torchvision import models
from torch import nn
from PIL import Image, ImageFilter

def preprocess_image(pil_im, resize_im=True):
    """
        Processes image for CNNs
    Args:
        PIL_img (PIL_img): PIL Image or numpy array to process
        resize_im (bool): Resize to 224 or not
    returns:
        im_as_var (torch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    #ensure or transform incoming image to PIL image
    if type(pil_im) != Image.Image:
        try:
            pil_im = Image.fromarray(pil_im)
        except Exception as e:
            print("could not transform PIL_img to a PIL Image object. Please check input.")

    # Resize image
    if resize_im:
        pil_im = pil_im.resize((224, 224), Image.ANTIALIAS)

    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var

def recreate_image(im_as_var):
    """
        Recreates images from a torch variable, sort of reverse preprocessing
    Args:
        im_as_var (torch variable): Image to recreate
    returns:
        recreated_im (numpy arr): Recreated image in array
    """
    reverse_mean = [-0.485, -0.456, -0.406]
    reverse_std = [1/0.229, 1/0.224, 1/0.225]
    recreated_im = copy.copy(im_as_var.data.numpy()[0])
    for c in range(3):
        recreated_im[c] /= reverse_std[c]
        recreated_im[c] -= reverse_mean[c]
    recreated_im[recreated_im > 1] = 1
    recreated_im[recreated_im < 0] = 0
    recreated_im = np.round(recreated_im * 255)

    recreated_im = np.uint8(recreated_im).transpose(1, 2, 0)
    return recreated_im

def save_image(im, path):
    """
        Saves a numpy matrix or PIL image as an image
    Args:
        im_as_arr (Numpy array): Matrix of shape DxWxH
        path (str): Path to the image
    """
    if torch.is_tensor(im):
        recreated_im = copy.copy(im.data.numpy()[0])
        im = np.uint8(recreated_im)
    if isinstance(im, (np.ndarray, np.generic)):
        im = format_np_output(im)
        im = Image.fromarray(im)
    im.save(path)

def format_np_output(np_arr):
    """
        This is a (kind of) bandaid fix to streamline saving procedure.
        It converts all the outputs to the same format which is 3xWxH
        with using sucecssive if clauses.
    Args:
        im_as_arr (Numpy array): Matrix of shape 1xWxH or WxH or 3xWxH
    """
    # Phase/Case 1: The np arr only has 2 dimensions
    # Result: Add a dimension at the beginning
    if len(np_arr.shape) == 2:
        np_arr = np.expand_dims(np_arr, axis=0)
    # Phase/Case 2: Np arr has only 1 channel (assuming first dim is channel)
    # Result: Repeat first channel and convert 1xWxH to 3xWxH
    if np_arr.shape[0] == 1:
        np_arr = np.repeat(np_arr, 3, axis=0)
    # Phase/Case 3: Np arr is of shape 3xWxH
    # Result: Convert it to WxHx3 in order to make it saveable by PIL
    if np_arr.shape[0] == 3:
        np_arr = np_arr.transpose(1, 2, 0)
    # Phase/Case 4: NP arr is normalized between 0-1
    # Result: Multiply with 255 and change type to make it saveable by PIL
    if np.max(np_arr) <= 1:
        np_arr = (np_arr*255).astype(np.uint8)
    return np_arr


# Non-Reguralized

In [ ]:
"""
Created on Thu Oct 26 14:19:44 2017
@author: Utku Ozbulak - github.com/utkuozbulak
"""

class ClassSpecificImageGeneration():
    """
        Produces an image that maximizes a certain class with gradient ascent
    """
    def __init__(self, model, target_class):
        self.mean = [-0.485, -0.456, -0.406]
        self.std = [1/0.229, 1/0.224, 1/0.225]
        self.model = model
        self.model.eval()
        self.target_class = target_class
        # Generate a random image
        self.created_image = np.uint8(np.random.uniform(0, 255, (224, 224, 3)))
        # Create the folder to export images if not exists
        if not os.path.exists('../generated/class_'+str(self.target_class)):
            os.makedirs('../generated/class_'+str(self.target_class))

    def generate(self, iterations=150):
        """Generates class specific image
        Keyword Arguments:
            iterations {int} -- Total iterations for gradient ascent (default: {150})
        Returns:
            np.ndarray -- Final maximally activated class image
        """
        initial_learning_rate = 6
        for i in range(1, iterations):
            # Process image and return variable
            self.processed_image = preprocess_image(self.created_image, False)

            # Define optimizer for the image
            optimizer = SGD([self.processed_image], lr=initial_learning_rate)
            # Forward
            output = self.model(self.processed_image)
            # Target specific class
            class_loss = -output[0, self.target_class]

            if i % 10 == 0 or i == iterations-1:
                print('Iteration:', str(i), 'Loss',
                      "{0:.2f}".format(class_loss.data.numpy()))
            # Zero grads
            self.model.zero_grad()
            # Backward
            class_loss.backward()
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(self.processed_image)
            if i % 10 == 0 or i == iterations-1:
                # Save image
                im_path = '/content/drive/MyDrive/DLSP/generated/class_'+str(self.target_class)+'/c_'+str(self.target_class)+'_'+'iter_'+str(i)+'.png'
                save_image(self.created_image, im_path)

        return self.processed_image

target_class = 2
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
######LOADING IN WEIGHTS#######
resume = ""
resume = "./BestModelSoFar/006.ckpt"
if resume:
    ckpt = torch.load(resume, map_location=torch.device('cpu'))
    model.load_state_dict(ckpt['net_state_dict'])


#target_class = 130  # Flamingo
#pretrained_model = models.alexnet(pretrained=True)
csig = ClassSpecificImageGeneration(model, target_class)
csig.generate()

NameError: ignored

# Reguralized

In [ ]:
use_cuda = torch.cuda.is_available()

class RegularizedClassSpecificImageGeneration():
    """
        Produces an image that maximizes a certain class with gradient ascent. Uses Gaussian blur, weight decay, and clipping. 
    """

    def __init__(self, model, target_class):
        self.mean = [-0.485, -0.456, -0.406]
        self.std = [1/0.229, 1/0.224, 1/0.225]
        self.model = model.cuda() if use_cuda else model
        self.model.eval()
        self.target_class = target_class
        # Generate a random image
        self.created_image = np.uint8(np.random.uniform(0, 255, (224, 224, 3)))
        # Create the folder to export images if not exists
        if not os.path.exists(f'/content/drive/MyDrive/DLSP/generated_R/class_{self.target_class}'):
            os.makedirs(f'/content/drive/MyDrive/DLSP/generated_R/class_{self.target_class}')

    def generate(self, iterations=150, blur_freq=4, blur_rad=1, wd=0.0001, clipping_value=0.1):
        """Generates class specific image with enhancements to improve image quality. 
        See https://arxiv.org/abs/1506.06579 for details on each argument's effect on output quality. 
        
        Play around with combinations of arguments. Besides the defaults, this combination has produced good images:
        blur_freq=6, blur_rad=0.8, wd = 0.05
        Keyword Arguments:
            iterations {int} -- Total iterations for gradient ascent (default: {150})
            blur_freq {int} -- Frequency of Gaussian blur effect, in iterations (default: {6})
            blur_rad {float} -- Radius for gaussian blur, passed to PIL.ImageFilter.GaussianBlur() (default: {0.8})
            wd {float} -- Weight decay value for Stochastic Gradient Ascent (default: {0.05})
            clipping_value {None or float} -- Value for gradient clipping (default: {0.1})
        
        Returns:
            np.ndarray -- Final maximally activated class image
        """
        initial_learning_rate = 6
        for i in range(1, iterations):
            # Process image and return variable

            #implement gaussian blurring every ith iteration
            #to improve output
            if i % blur_freq == 0:
                self.processed_image = preprocess_and_blur_image(
                    self.created_image, False, blur_rad)
            else:
                self.processed_image = preprocess_and_blur_image(
                    self.created_image, False)

            if use_cuda:
                self.processed_image = self.processed_image.cuda()

            # Define optimizer for the image - use weight decay to add regularization
            # in SGD, wd = 2 * L2 regularization (https://bbabenko.github.io/weight-decay/)
            optimizer = SGD([self.processed_image],
                            lr=initial_learning_rate, weight_decay=wd)
            # Forward
            output = self.model(self.processed_image)
            # Target specific class
            class_loss = -output[0, self.target_class]

            if i in np.linspace(0, iterations, 10, dtype=int):
                print('Iteration:', str(i), 'Loss',
                      "{0:.2f}".format(class_loss.data.cpu().numpy()))
            # Zero grads
            self.model.zero_grad()
            # Backward
            class_loss.backward()

            if clipping_value:
                torch.nn.utils.clip_grad_norm(
                    self.model.parameters(), clipping_value)
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(self.processed_image.cpu())

            if i in np.linspace(0, iterations, 10, dtype=int):
                # Save image
                im_path = f'/content/drive/MyDrive/DLSP/generated_R/class_{self.target_class}/c_{self.target_class}_iter_{i}_loss_{class_loss.data.cpu().numpy()}.jpg'
                save_image(self.created_image, im_path)

        #save final image
        im_path = f'/content/drive/MyDrive/DLSP/generated_R/class_{self.target_class}/c_{self.target_class}_iter_{i}_loss_{class_loss.data.cpu().numpy()}.jpg'
        save_image(self.created_image, im_path)

        #write file with regularization details
        with open(f'/content/drive/MyDrive/DLSP/generated_R/class_{self.target_class}/run_details.txt', 'w') as f:
            f.write(f'Iterations: {iterations}\n')
            f.write(f'Blur freq: {blur_freq}\n')
            f.write(f'Blur radius: {blur_rad}\n')
            f.write(f'Weight decay: {wd}\n')
            f.write(f'Clip value: {clipping_value}\n')

        #rename folder path with regularization details for easy access
        os.rename(f'/content/drive/MyDrive/DLSP/generated_R/class_{self.target_class}',
                  f'/content/drive/MyDrive/DLSP/generated_R/class_{self.target_class}_blurfreq_{blur_freq}_blurrad_{blur_rad}_wd_{wd}_CV{clipping_value}')
        return self.processed_image


def preprocess_and_blur_image(pil_im, resize_im=True, blur_rad=None):
    """
        Processes image with optional Gaussian blur for CNNs
    Args:
        PIL_img (PIL_img): PIL Image or numpy array to process
        resize_im (bool): Resize to 224 or not
        blur_rad (int): Pixel radius for Gaussian blurring (default = None)
    returns:
        im_as_var (torch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    #ensure or transform incoming image to PIL image
    if type(pil_im) != Image.Image:
        try:
            pil_im = Image.fromarray(pil_im)
        except Exception as e:
            print(
                "could not transform PIL_img to a PIL Image object. Please check input.")

    # Resize image
    if resize_im:
        pil_im.thumbnail((224, 224))

    #add gaussin blur to image
    if blur_rad:
        pil_im = pil_im.filter(ImageFilter.GaussianBlur(blur_rad))

    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    if use_cuda:
        im_as_var = Variable(im_as_ten.cuda(), requires_grad=True)
    else:
        im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var

target_class = 0
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
######LOADING IN WEIGHTS#######
resume = ""
resume = "/content/drive/MyDrive/DLSP/models/FinalModels/Train2/018.ckpt"
if resume:
    ckpt = torch.load(resume, map_location=torch.device('cpu'))
    model.load_state_dict(ckpt['net_state_dict'])
#target_class = 130  # Flamingo
#pretrained_model = models.alexnet(pretrained=True)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
for target_class in [4,7]:
  for blur_freq in [3,4,5,6,7]:
      for blur_rad in [1]:
          for wd in [0.1, 0.05, 0.005, 0.0001]:
              for clipping_value in [0.05, 0.1, 0.15]:
                  gen = RegularizedClassSpecificImageGeneration(model, target_class)
                  gen.generate(iterations=150, blur_freq=blur_freq, blur_rad=blur_rad, wd=wd, clipping_value=clipping_value)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:71: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Iteration: 16 Loss -1.23
Iteration: 33 Loss -5.08
Iteration: 50 Loss -14.00
Iteration: 66 Loss -8.10
Iteration: 83 Loss -19.19
Iteration: 100 Loss -9.12
Iteration: 116 Loss -18.74
Iteration: 133 Loss -10.60
Iteration: 16 Loss -3.87
Iteration: 33 Loss -5.33
Iteration: 50 Loss -14.48
Iteration: 66 Loss -6.77
Iteration: 83 Loss -16.67
Iteration: 100 Loss -10.39
Iteration: 116 Loss -13.19
Iteration: 133 Loss -10.13
Iteration: 16 Loss -5.31
Iteration: 33 Loss -4.03
Iteration: 50 Loss -12.99
Iteration: 66 Loss -6.17
Iteration: 83 Loss -18.25
Iteration: 100 Loss -9.55
Iteration: 116 Loss -18.36
Iteration: 133 Loss -8.60
Iteration: 16 Loss -9.61
Iteration: 33 Loss -36.96
Iteration: 50 Loss -43.85
Iteration: 66 Loss -34.51
Iteration: 83 Loss -44.95
Iteration: 100 Loss -13.81
Iteration: 116 Loss -46.72
Iteration: 133 Loss -15.80
Iteration: 16 Loss -9.56
Iteration: 33 Loss -38.58
Iteration: 50 Loss -43.13
Iteration: 66 Loss -39.88
Iteration: 83 Loss -51.24
Iteration: 100 Loss -18.97
Iteration: 11

# Reguralized with mean images

## Loading meanimages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
run_mean_images = []
combinations = [("Food", 0), ("Food", 2), ("Medical", 1), ("Medical", 4)]
for comb in combinations:
  # Loading mean images
  stringer = "/content/drive/MyDrive/DLSP/Drna-master/MeanImages/"+comb[0]+".npy"
  MeanImagesAll = np.load(stringer, allow_pickle = True)

  selected_cluster = comb[1]
  selected_mean_image = MeanImagesAll[selected_cluster,:,:]
  selected_mean_image.shape

  #Reshaping into right shape
  selected_mean_image_raw = np.swapaxes(np.swapaxes(selected_mean_image,0,1),1,2)
  # Rescaling
  selected_mean_image = np.clip(selected_mean_image_raw,0,1)
  run_mean_images.append(selected_mean_image)


## Generating images

In [ ]:
use_cuda = torch.cuda.is_available()

class RegularizedClassSpecificImageGeneration():
    """
        Produces an image that maximizes a certain class with gradient ascent. Uses Gaussian blur, weight decay, and clipping. 
    """

    def __init__(self, model, target_class, selected_mean_image, cluster):
        self.mean = [-0.485, -0.456, -0.406]
        self.std = [1/0.229, 1/0.224, 1/0.225]
        self.model = model.cuda() if use_cuda else model
        self.model.eval()
        self.target_class = target_class
        self.cluster = cluster
        # Generate a random image
        self.created_image = np.uint8(np.random.uniform(0, 255, (224, 224, 3)))
        self.created_image = np.uint8(selected_mean_image*255)
        # Create the folder to export images if not exists
        if not os.path.exists(f'/content/drive/MyDrive/DLSP/generated_R1/class_{self.target_class}'):
            os.makedirs(f'/content/drive/MyDrive/DLSP/generated_R1/class_{self.target_class}')

    def generate(self, iterations=150, blur_freq=4, blur_rad=1, wd=0.01, clipping_value=0.1):
        """Generates class specific image with enhancements to improve image quality. 
        See https://arxiv.org/abs/1506.06579 for details on each argument's effect on output quality. 
        #ORIGINAL iterations=150, blur_freq=4, blur_rad=1, wd=0.0001, clipping_value=0.1
        Play around with combinations of arguments. Besides the defaults, this combination has produced good images:
        blur_freq=6, blur_rad=0.8, wd = 0.05
        Keyword Arguments:
            iterations {int} -- Total iterations for gradient ascent (default: {150})
            blur_freq {int} -- Frequency of Gaussian blur effect, in iterations (default: {6})
            blur_rad {float} -- Radius for gaussian blur, passed to PIL.ImageFilter.GaussianBlur() (default: {0.8})
            wd {float} -- Weight decay value for Stochastic Gradient Ascent (default: {0.05})
            clipping_value {None or float} -- Value for gradient clipping (default: {0.1})
        
        Returns:
            np.ndarray -- Final maximally activated class image
        """
        initial_learning_rate = 6
        for i in range(1, iterations):
            # Process image and return variable

            #implement gaussian blurring every ith iteration
            #to improve output
            if i % blur_freq == 0:
                self.processed_image = preprocess_and_blur_image(
                    self.created_image, False, blur_rad)
            else:
                self.processed_image = preprocess_and_blur_image(
                    self.created_image, False)

            if use_cuda:
                self.processed_image = self.processed_image.cuda()

            # Define optimizer for the image - use weight decay to add regularization
            # in SGD, wd = 2 * L2 regularization (https://bbabenko.github.io/weight-decay/)
            optimizer = SGD([self.processed_image],
                            lr=initial_learning_rate, weight_decay=wd)
            # Forward
            output = self.model(self.processed_image)
            # Target specific class
            class_loss = -output[0, self.target_class]

            if i in np.linspace(0, iterations, 10, dtype=int):
                print('Iteration:', str(i), 'Loss',
                      "{0:.2f}".format(class_loss.data.cpu().numpy()))
            # Zero grads
            self.model.zero_grad()
            # Backward
            class_loss.backward()

            if clipping_value:
                torch.nn.utils.clip_grad_norm(
                    self.model.parameters(), clipping_value)
            # Update image
            optimizer.step()
            # Recreate image
            self.created_image = recreate_image(self.processed_image.cpu())

            if i in np.linspace(0, iterations, 10, dtype=int):
                # Save image
                im_path = f'/content/drive/MyDrive/DLSP/generated_R1/class_{self.target_class}/c_{self.target_class}_iter_{i}_loss_{class_loss.data.cpu().numpy()}.jpg'
                save_image(self.created_image, im_path)

        #save final image
        im_path = f'/content/drive/MyDrive/DLSP/generated_R1/class_{self.target_class}/c_{self.target_class}_iter_{i}_loss_{class_loss.data.cpu().numpy()}.jpg'
        save_image(self.created_image, im_path)

        #write file with regularization details
        with open(f'/content/drive/MyDrive/DLSP/generated_R1/class_{self.target_class}/run_details.txt', 'w') as f:
            f.write(f'Iterations: {iterations}\n')
            f.write(f'Blur freq: {blur_freq}\n')
            f.write(f'Blur radius: {blur_rad}\n')
            f.write(f'Weight decay: {wd}\n')
            f.write(f'Clip value: {clipping_value}\n')

        #rename folder path with regularization details for easy access
        os.rename(f'/content/drive/MyDrive/DLSP/generated_R1/class_{self.target_class}',
                  f'/content/drive/MyDrive/DLSP/generated_R1/class_{self.target_class}_blurfreq_{blur_freq}_blurrad_{blur_rad}_wd{wd}_CV{clipping_value}_cluster_{self.cluster}')
        return self.processed_image


def preprocess_and_blur_image(pil_im, resize_im=True, blur_rad=None):
    """
        Processes image with optional Gaussian blur for CNNs
    Args:
        PIL_img (PIL_img): PIL Image or numpy array to process
        resize_im (bool): Resize to 224 or not
        blur_rad (int): Pixel radius for Gaussian blurring (default = None)
    returns:
        im_as_var (torch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    #ensure or transform incoming image to PIL image
    if type(pil_im) != Image.Image:
        try:
            pil_im = Image.fromarray(pil_im)
        except Exception as e:
            print(
                "could not transform PIL_img to a PIL Image object. Please check input.")

    # Resize image
    if resize_im:
        pil_im.thumbnail((224, 224))

    #add gaussin blur to image
    if blur_rad:
        pil_im = pil_im.filter(ImageFilter.GaussianBlur(blur_rad))

    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    if use_cuda:
        im_as_var = Variable(im_as_ten.cuda(), requires_grad=True)
    else:
        im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var

target_class = 7
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
######LOADING IN WEIGHTS#######
resume = ""
resume = "/content/drive/MyDrive/DLSP/models/FinalModels/Train2/018.ckpt"
if resume:
    ckpt = torch.load(resume, map_location=torch.device('cpu'))
    model.load_state_dict(ckpt['net_state_dict'])
#target_class = 130  # Flamingo
#pretrained_model = models.alexnet(pretrained=True)
#csig = RegularizedClassSpecificImageGeneration(model, target_class, selected_mean_image)
#csig.generate()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
combinations[2:]

[('Medical', 1), ('Medical', 4)]

In [ ]:
for i,comb in enumerate(combinations[3:]):
  if comb[0] == "Food":
    target_class = 4
  else:
    target_class = 7
  for blur_freq in [3,4,5,6,7]:
      for blur_rad in [1]:
          for wd in [0.1, 0.05, 0.005, 0.0001]:
              for clipping_value in [0.05, 0.1, 0.15]:
                  gen = RegularizedClassSpecificImageGeneration(model, target_class, run_mean_images[i], comb[1])
                  gen.generate(iterations=150, blur_freq=blur_freq, blur_rad=blur_rad, wd=wd, clipping_value=clipping_value)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


Iteration: 16 Loss -1.84
Iteration: 33 Loss -0.32
Iteration: 50 Loss -3.62
Iteration: 66 Loss -0.18
Iteration: 83 Loss -1.61
Iteration: 100 Loss 0.25
Iteration: 116 Loss -4.68
Iteration: 133 Loss 0.68
Iteration: 16 Loss -1.84
Iteration: 33 Loss -0.32
Iteration: 50 Loss -3.62
Iteration: 66 Loss -0.18
Iteration: 83 Loss -1.61
Iteration: 100 Loss 0.25
Iteration: 116 Loss -4.68
Iteration: 133 Loss 0.68
Iteration: 16 Loss -1.84
Iteration: 33 Loss -0.32
Iteration: 50 Loss -3.62
Iteration: 66 Loss -0.18
Iteration: 83 Loss -1.61
Iteration: 100 Loss 0.25
Iteration: 116 Loss -4.68
Iteration: 133 Loss 0.68
Iteration: 16 Loss -2.31
Iteration: 33 Loss -26.73
Iteration: 50 Loss -26.65
Iteration: 66 Loss -31.77
Iteration: 83 Loss -21.95
Iteration: 100 Loss -11.11
Iteration: 116 Loss -41.65
Iteration: 133 Loss -15.26
Iteration: 16 Loss -2.31
Iteration: 33 Loss -26.73
Iteration: 50 Loss -26.65
Iteration: 66 Loss -31.77
Iteration: 83 Loss -21.95
Iteration: 100 Loss -11.11
Iteration: 116 Loss -41.65
Iter

KeyboardInterrupt: ignored

In [ ]:
https://github.com/utkuozbulak/pytorch-cnn-visualizations/tree/master/src

# Reguralized with GAN

## Defining Generator-network (from a trained ACGAN)

In [4]:
def initialize_weights(net):
    for m in net.modules():
        if isinstance(m, nn.Conv2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.ConvTranspose2d):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.02)
            m.bias.data.zero_()

class generator(nn.Module):
    # Network Architecture is exactly same as in infoGAN (https://arxiv.org/abs/1606.03657)
    # Architecture : FC1024_BR-FC7x7x128_BR-(64)4dc2s_BR-(1)4dc2s_S
    def __init__(self, input_dim=100, output_dim=1, input_size=32, class_num=10):
        super(generator, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.input_size = input_size
        self.class_num = class_num

        self.fc = nn.Sequential(
            nn.Linear(self.input_dim + self.class_num, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),
            nn.Linear(1024, 128 * (self.input_size // 4) * (self.input_size // 4)),
            nn.BatchNorm1d(128 * (self.input_size // 4) * (self.input_size // 4)),
            nn.ReLU(),
        )
        self.deconv = nn.Sequential(
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, self.output_dim, 4, 2, 1),
            nn.Tanh(),
        )
        initialize_weights(self)

    def forward(self, input, label):
        x = torch.cat([input, label], 1)
        x = self.fc(x)
        x = x.view(-1, 128, (self.input_size // 4), (self.input_size // 4))
        x = self.deconv(x)

        return x


## Generate AM with GAN

In [16]:
use_cuda = torch.cuda.is_available()
use_cuda = False

class RegularizedClassSpecificImageGeneration():
    """
        Produces an image that maximizes a certain class with gradient ascent. Uses Gaussian blur, weight decay, and clipping. 
    """

    def __init__(self, model, target_class):
        #self.mean = [-0.485, -0.456, -0.406]
        #self.std = [1/0.229, 1/0.224, 1/0.225]
        self.model = model.cuda() if use_cuda else model
        self.model.eval()
        self.target_class = target_class
        # Generate a random image
        self.created_image = np.uint8(np.random.uniform(0, 255, (224, 224, 3)))
        #self.created_image = np.uint8(selected_mean_image*255)
        #######BEGIN GAN######
        # Generate class-vector for GAN:
        self.y = torch.tensor([target_class,target_class])
        self.y_vec_ = torch.zeros((2, 10)).scatter_(1, self.y.type(torch.LongTensor).unsqueeze(1), 1)
        # Generate random z-vector
        #self.z = torch.rand((1, 62))
        self.z = torch.rand((2, 62))
        #Init GAN
        self.G = generator(input_dim=62, output_dim=3, input_size=112)
        self.G.load_state_dict(torch.load("/content/drive/MyDrive/DLSP/models/FinalModels/Gan/ACGAN_G.pkl",map_location=torch.device('cpu')))
        self.G.eval()
        #######END GAN########
        # Create the folder to export images if not exists
        if not os.path.exists(f'/content/drive/MyDrive/DLSP/generated_R2/class_{self.target_class}'):
            os.makedirs(f'/content/drive/MyDrive/DLSP/generated_R2/class_{self.target_class}')

    def generate(self, iterations=150, blur_freq=4, blur_rad=1, wd=0.0001, clipping_value=0.1):
        """Generates class specific image with enhancements to improve image quality. 
        See https://arxiv.org/abs/1506.06579 for details on each argument's effect on output quality. 
        #ORIGINAL iterations=150, blur_freq=4, blur_rad=1, wd=0.0001, clipping_value=0.1
        Play around with combinations of arguments. Besides the defaults, this combination has produced good images:
        blur_freq=6, blur_rad=0.8, wd = 0.05
        Keyword Arguments:
            iterations {int} -- Total iterations for gradient ascent (default: {150})
            blur_freq {int} -- Frequency of Gaussian blur effect, in iterations (default: {6})
            blur_rad {float} -- Radius for gaussian blur, passed to PIL.ImageFilter.GaussianBlur() (default: {0.8})
            wd {float} -- Weight decay value for Stochastic Gradient Ascent (default: {0.05})
            clipping_value {None or float} -- Value for gradient clipping (default: {0.1})
        
        Returns:
            np.ndarray -- Final maximally activated class image
        """
        initial_learning_rate = 6
        for i in range(1, iterations):
            #Manipulating image
            self.img = self.G(self.z,self.y_vec_)
            self.img = self.img[0,:,:,:]
            #print(self.img.shape)
            self.img_np = self.img.squeeze().detach().numpy()
            self.img_np = np.swapaxes(np.swapaxes(self.img_np,0,1),1,2)
            self.img_np = np.uint8(np.clip(self.img_np,0,1)*255)
            self.img_pil = Image.fromarray(self.img_np)
            self.img_pil = self.img_pil.resize((224,224),Image.BICUBIC)
            self.im_as_arr = np.float32(self.img_pil)
            self.im_as_arr = self.im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
            # Convert to float tensor
            self.im_as_ten = torch.from_numpy(self.im_as_arr).float()
            # Add one more channel to the beginning. Tensor shape = 1,3,224,224
            self.created_image = self.im_as_ten.unsqueeze_(0)
            self.x0 = self.created_image.clone()
            self.created_image = Variable(self.created_image,requires_grad=True)


            #implement gaussian blurring every ith iteration
            #to improve output
            '''
            if i % blur_freq == 0:
                self.processed_image = preprocess_and_blur_image(
                    self.created_image, False, blur_rad)
            else:
                self.processed_image = preprocess_and_blur_image(
                    self.created_image, False)
            '''
            
            if use_cuda:
                self.created_image = self.created_image.cuda()
                print("using cuda")
            # Define optimizer for the image - use weight decay to add regularization
            # in SGD, wd = 2 * L2 regularization (https://bbabenko.github.io/weight-decay/)
            optimizer = SGD([self.created_image],
                            lr=initial_learning_rate)
            # Forward
            output = self.model(self.created_image)
            # Target specific class
            class_loss = -output[0, self.target_class]

            if i in np.linspace(0, iterations, 10, dtype=int):
                print('Iteration:', str(i), 'Loss',
                      "{0:.2f}".format(class_loss.data.cpu().numpy()))
                #print(self.z)
            # Zero grads
            self.model.zero_grad()
            # Backward
            class_loss.backward()
            '''
            if clipping_value:
                torch.nn.utils.clip_grad_norm(
                    self.model.parameters(), clipping_value)
            '''
            # Update image
            optimizer.step()
            
            #MAKE GENERATOR STEP
            self.z = generator_step(self.G, self.created_image, self.z, self.y_vec_, initial_learning_rate)


            # Recreate image
            #self.created_image = recreate_image(self.created_image.cpu())
            
            if i in np.linspace(0, iterations, 10, dtype=int):
                # Save image
                im_path = f'/content/drive/MyDrive/DLSP/generated_R2/class_{self.target_class}/c_{self.target_class}_iter_{i}_loss_{class_loss.data.cpu().numpy()}.jpg'
                save_image(self.created_image, im_path)

        #save final image
        im_path = f'/content/drive/MyDrive/DLSP/generated_R2/class_{self.target_class}/c_{self.target_class}_iter_{i}_loss_{class_loss.data.cpu().numpy()}.jpg'
        save_image(self.created_image, im_path)

        #write file with regularization details
        with open(f'/content/drive/MyDrive/DLSP/generated_R2/class_{self.target_class}/run_details.txt', 'w') as f:
            f.write(f'Iterations: {iterations}\n')
            f.write(f'Blur freq: {blur_freq}\n')
            f.write(f'Blur radius: {blur_rad}\n')
            f.write(f'Weight decay: {wd}\n')
            f.write(f'Clip value: {clipping_value}\n')

        #rename folder path with regularization details for easy access
        os.rename(f'/content/drive/MyDrive/DLSP/generated_R2/class_{self.target_class}',
                  f'/content/drive/MyDrive/DLSP/generated_R2/class_{self.target_class}_blurfreq_{blur_freq}_blurrad_{blur_rad}_wd{wd}')
        return self.created_image


def generator_step(G, x, oldz, y,lr):
    z = oldz
    z = Variable(z, requires_grad = True)
    #splitting z
    #z0 = z[0,:].unsqueeze(0)
    #z1 = z[1,:].unsqueeze(0)
    #setting optimizer
    #z0 = Variable(z0, requires_grad = True)
    optimizer = torch.optim.SGD([z], lr=0.00001)
    # run z through generator
    #x0 = G(torch.concat([z0,z1]),y)
    x0 = G(z,y)
    #fetching first image (batch of 2)
    img = x0[0,:,:,:]
    #Do image manipulation to input x get same size as output from generator (from 224 -> 112)
    # Same manipulation as getting output from generator to same size as inpu to CNN (from 112 -> 224)
    x_np = x.squeeze().detach().numpy()
    x_np = np.swapaxes(np.swapaxes(x_np,0,1),1,2)
    x_np = np.uint8(np.clip(x_np,0,1)*255)
    x_pil = Image.fromarray(x_np)
    x_pil = x_pil.resize((112,112),Image.BICUBIC)
    x_as_arr = np.float32(x_pil)
    x_as_arr = x_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Convert to float tensor
    x_as_ten = torch.from_numpy(x_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    created_x = x_as_ten.unsqueeze_(0)
    x = created_x
    loss= ((img-x)**2).sum()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    #Making sure Z-vec is between 0,1
    #z0 = torch.clamp(z0,0,1)
    z = torch.clamp(z,0,1)
    return z
    
def preprocess_and_blur_image(pil_im, resize_im=True, blur_rad=None):
    """
        Processes image with optional Gaussian blur for CNNs
    Args:
        PIL_img (PIL_img): PIL Image or numpy array to process
        resize_im (bool): Resize to 224 or not
        blur_rad (int): Pixel radius for Gaussian blurring (default = None)
    returns:
        im_as_var (torch variable): Variable that contains processed float tensor
    """
    # mean and std list for channels (Imagenet)
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    #ensure or transform incoming image to PIL image
    if type(pil_im) != Image.Image:
        try:
            pil_im = Image.fromarray(pil_im)
        except Exception as e:
            print(
                "could not transform PIL_img to a PIL Image object. Please check input.")

    # Resize image
    if resize_im:
        pil_im.thumbnail((224, 224))

    #add gaussin blur to image
    if blur_rad:
        pil_im = pil_im.filter(ImageFilter.GaussianBlur(blur_rad))

    im_as_arr = np.float32(pil_im)
    im_as_arr = im_as_arr.transpose(2, 0, 1)  # Convert array to D,W,H
    # Normalize the channels
    for channel, _ in enumerate(im_as_arr):
        im_as_arr[channel] /= 255
        im_as_arr[channel] -= mean[channel]
        im_as_arr[channel] /= std[channel]
    # Convert to float tensor
    im_as_ten = torch.from_numpy(im_as_arr).float()
    # Add one more channel to the beginning. Tensor shape = 1,3,224,224
    im_as_ten.unsqueeze_(0)
    # Convert to Pytorch variable
    if use_cuda:
        im_as_var = Variable(im_as_ten.cuda(), requires_grad=True)
    else:
        im_as_var = Variable(im_as_ten, requires_grad=True)
    return im_as_var

#target_class = 0
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)
######LOADING IN WEIGHTS#######
resume = ""
resume = "/content/drive/MyDrive/DLSP/models/FinalModels/Train2/018.ckpt"
if resume:
    ckpt = torch.load(resume, map_location=torch.device('cpu'))
    model.load_state_dict(ckpt['net_state_dict'])
#target_class = 130  # Flamingo
#pretrained_model = models.alexnet(pretrained=True)
#csig = RegularizedClassSpecificImageGeneration(model, target_class)
#csig.generate()

In [17]:
for target_class in [0,1,2,3,4,5,6,7,8,9]:
  csig = RegularizedClassSpecificImageGeneration(model, target_class)
  csig.generate()

Iteration: 16 Loss -157.02
Iteration: 33 Loss -165.71
Iteration: 50 Loss -190.93
Iteration: 66 Loss -170.86
Iteration: 83 Loss -167.97
Iteration: 100 Loss -174.19
Iteration: 116 Loss -165.87
Iteration: 133 Loss -179.31
Iteration: 16 Loss -326.62
Iteration: 33 Loss -326.55
Iteration: 50 Loss -326.32
Iteration: 66 Loss -326.24
Iteration: 83 Loss -326.24
Iteration: 100 Loss -326.24
Iteration: 116 Loss -326.24
Iteration: 133 Loss -326.24
Iteration: 16 Loss 45.58
Iteration: 33 Loss 47.90
Iteration: 50 Loss 44.74
Iteration: 66 Loss 45.87
Iteration: 83 Loss 35.92
Iteration: 100 Loss 45.02
Iteration: 116 Loss 47.69
Iteration: 133 Loss 44.85
Iteration: 16 Loss -155.87
Iteration: 33 Loss -158.87
Iteration: 50 Loss -158.49
Iteration: 66 Loss -158.90
Iteration: 83 Loss -159.01
Iteration: 100 Loss -158.75
Iteration: 116 Loss -157.13
Iteration: 133 Loss -157.07
Iteration: 16 Loss 128.57
Iteration: 33 Loss 128.51
Iteration: 50 Loss 128.66
Iteration: 66 Loss 128.62
Iteration: 83 Loss 128.33
Iteration: